<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-19 10:42:07
-------------------
qualified stocks: 95
with latest results: 26
still star stocks: 16


-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  2.11 L
Current:  1.29 C
-------------------
Today PnL: 88.35 K (0.69%)
Current PnL: -14.91 L (-10.57%)
CY Booked + Current PnL: -11.78 L (-8.36%)
-------------------
Total profit:  4.51 L
Total loss:  -19.43 L
-------------------
Total Booked + Current PnL: 15.20 L (13.33%)
Total Booked PnL: 30.11 L (26.4%)
Curr Year Booked PnL: 3.13 L (2.43%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.07 C
Est FTT PnL: 77.62 L (60.18%)
-------------------
Est LTT:  2.68 C
Est LTT PnL: 1.39 C (107.85%)
Deployed:  1.14 C
Current:  1.29 C
CAGR/XIRR %: 8.85%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,RELIANCE,1288.53,1526.00,6.12,X-LC,81.05,228594.0,25006.0,12504.0,-0.66,12.28,5.47,18.43,157.0,2.00,1.80,24.52,XY25,NTT,REFINERIES
69,SBILIFE,1496.49,1924.99,-0.07,H-LC,72.63,148562.0,22857.0,13133.0,-0.27,18.18,8.84,28.63,147.0,1.74,1.17,32.35,AR,ATH,INSURANCE
56,PIDILITIND,2504.06,3576.00,-16.68,X-LC,35.26,92328.0,17206.0,14948.0,-0.44,22.90,16.19,42.81,120.0,1.15,0.73,15.79,X40,BTT,CHEMICALS
6,ANGELONE,2328.67,3033.00,11.28,H-SC,88.42,181344.0,29980.0,15795.0,-0.38,19.81,8.71,30.25,111.0,1.90,1.43,43.53,X40N,NTT,FINANCE
70,SBIN,760.30,863.00,-13.45,M-LC,23.16,211324.0,9844.0,17371.0,0.68,4.89,8.22,13.51,60.0,0.57,1.67,18.10,XY25,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TITAGARH,1097.23,1548.00,-3.47,H-SC,98.95,169645.0,-22370.0,101261.0,6.19,-11.65,59.69,41.08,95.0,-0.22,1.34,43.58,XY24,NTT,ENGINEERING
89,VALIANTORG,512.64,838.00,-340.80,H-SC,61.05,102284.0,-31002.0,115591.0,4.99,-23.26,113.01,63.47,161.0,-0.27,0.81,71.45,XR,NTT,CHEMICALS
30,HAPPSTMNDS,919.80,1491.84,-21.84,M-SC,66.32,65835.0,-30744.0,90806.0,3.70,-31.83,137.93,62.19,28.0,-0.34,0.52,15.44,AR,ATH,IT
88,VAIBHAVGBL,350.21,521.00,68.60,X-SC,89.47,112432.0,-39559.0,113680.0,3.27,-26.03,101.11,48.77,50.0,-0.35,0.89,29.14,XR,NTT,JEWELLERY
84,TRIDENT,37.35,48.00,55.38,M-SC,77.89,74643.0,-17612.0,43920.0,2.86,-19.09,58.84,28.51,204.0,-0.40,0.59,25.81,XR,NTT,TEXTILES


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1022.60,1486.00,1.39,X-MC,3.16,178828.0,23393.0,47050.0,-2.05,15.05,26.31,45.32,201.0,0.50,1.41,21.68,XY24,NTT,HEALTHCARE
35,HONAUT,42646.15,58518.52,-22.02,X-MC,75.79,114255.0,-13683.0,61298.0,-1.89,-10.70,53.65,37.22,73.0,-0.22,0.90,16.65,X40N,ATH,ELECTRICAL
41,INFY,1432.81,2275.00,-12.42,X-LC,69.47,217924.0,18763.0,98306.0,-1.39,9.42,45.11,58.78,10.0,0.19,1.72,13.78,X40,BTT,IT
76,STARHEALTH,551.56,761.00,-1.20,M-SC,84.21,161535.0,-38681.0,114706.0,-1.31,-19.32,71.01,37.97,62.0,-0.34,1.27,29.70,XY24,NTT,INSURANCE
75,SONACOMS,578.05,1006.00,-27.37,M-MC,95.79,93844.0,-7315.0,82207.0,-1.22,-7.23,87.60,74.03,49.0,-0.09,0.74,31.89,AR,BTT,AUTO


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TATAMOTORS,776.02,1065.00,-51.41,M-LC,92.63,194603.0,-11042.0,87630.0,0.50,-5.37,45.03,37.24,1.0,-0.13,1.53,26.67,XY24,NTT,AUTO
38,INDIAMART,2311.97,4911.36,-51.65,H-SC,53.68,104632.0,2905.0,111464.0,1.18,2.86,106.53,112.43,2.0,0.03,0.82,23.49,AR,ATH,MISC
2,ABB,5319.37,8953.12,-35.69,H-LC,50.53,147125.0,14141.0,76696.0,1.12,10.63,52.13,68.31,4.0,0.18,1.16,19.97,AR,ATH,ELECTRICAL
82,TCS,3928.29,4998.00,-15.52,X-LC,57.89,190685.0,-21443.0,79211.0,-0.85,-10.11,41.54,27.23,5.0,-0.27,1.50,9.27,X40,BTT,IT
49,LICI,777.74,983.00,-13.28,H-LC,52.63,192886.0,18672.0,27313.0,0.48,10.72,14.16,26.39,6.0,0.68,1.52,18.09,XY25,NTT,INSURANCE


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DMART,3484.00,5112.00,-17.75,X-LC,13.68,101232.0,17616.0,21451.0,0.68,21.07,21.19,46.73,146.0,0.82,0.80,23.91,AR,NTT,FMCG
56,PIDILITIND,2504.06,3576.00,-16.68,X-LC,35.26,92328.0,17206.0,14948.0,-0.44,22.90,16.19,42.81,120.0,1.15,0.73,15.79,X40,BTT,CHEMICALS
57,POLYCAB,5005.92,7541.35,9.62,H-MC,87.37,171514.0,31348.0,39637.0,0.51,22.37,23.11,50.65,55.0,0.79,1.35,30.90,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,-1.13,M-LC,83.16,191608.0,40402.0,25254.0,-0.63,26.72,13.18,43.43,37.0,1.60,1.51,32.81,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,46.32,M-SC,73.68,151534.0,13956.0,80449.0,1.97,10.14,53.09,68.62,79.0,0.17,1.19,31.37,X40N,NTT,TEXTILES
6,ANGELONE,2328.67,3033.00,11.28,H-SC,88.42,181344.0,29980.0,15795.0,-0.38,19.81,8.71,30.25,111.0,1.90,1.43,43.53,X40N,NTT,FINANCE
57,POLYCAB,5005.92,7541.35,9.62,H-MC,87.37,171514.0,31348.0,39637.0,0.51,22.37,23.11,50.65,55.0,0.79,1.35,30.90,X40N,ATH,CABLES
33,HEROMOTOCO,4311.81,6039.03,1.43,H-MC,85.26,154476.0,3563.0,56894.0,1.57,2.36,36.83,40.06,14.0,0.06,1.22,26.35,AR,ATH,AUTO
28,GILLETTE,8109.44,10330.69,-0.07,H-SC,76.84,147500.0,9640.0,28128.0,0.77,6.99,19.07,27.39,68.0,0.34,1.16,29.12,X40,ATH,FMCG


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,-2.00,H-SC,18.95,110292.0,8288.0,36738.0,1.24,8.13,33.31,44.14,22.0,0.23,0.87,25.69,XR,ATH,MISC
94,WIPRO,243.46,420.00,-8.35,M-LC,60.00,157387.0,6442.0,103010.0,-0.18,4.27,65.45,72.51,52.0,0.06,1.24,18.50,XR,NTT,IT
33,HEROMOTOCO,4311.81,6039.03,1.43,H-MC,85.26,154476.0,3563.0,56894.0,1.57,2.36,36.83,40.06,14.0,0.06,1.22,26.35,AR,ATH,AUTO
12,BANKINDIA,113.49,190.00,-31.87,M-MC,16.32,133237.0,4085.0,82980.0,2.41,3.16,62.28,67.42,41.0,0.05,1.05,29.58,XR,NTT,BANKS
38,INDIAMART,2311.97,4911.36,-51.65,H-SC,53.68,104632.0,2905.0,111464.0,1.18,2.86,106.53,112.43,2.0,0.03,0.82,23.49,AR,ATH,MISC


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,VALIANTORG,512.64,838.00,-340.80,H-SC,61.05,102284.0,-31002.0,115591.0,4.99,-23.26,113.01,63.47,161.0,-0.27,0.81,71.45,XR,NTT,CHEMICALS
86,UJJIVANSFB,52.77,60.00,38.86,M-SC,30.53,120852.0,-21627.0,41150.0,0.38,-15.18,34.05,13.70,214.0,-0.53,0.95,42.64,X40N,NTT,BANKS
12,BANKINDIA,113.49,190.00,-31.87,M-MC,16.32,133237.0,4085.0,82980.0,2.41,3.16,62.28,67.42,41.0,0.05,1.05,29.58,XR,NTT,BANKS
69,SBILIFE,1496.49,1924.99,-0.07,H-LC,72.63,148562.0,22857.0,13133.0,-0.27,18.18,8.84,28.63,147.0,1.74,1.17,32.35,AR,ATH,INSURANCE
87,UNIONBANK,123.87,163.00,-12.58,M-LC,49.47,153813.0,12973.0,31516.0,1.70,9.21,20.49,31.59,48.0,0.41,1.21,33.61,XY24,NTT,BANKS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,VIPIND,488.80,718.00,-890.96,H-SC,100.00,75338.0,-19978.0,64670.0,1.34,-20.96,85.84,46.89,82.0,-0.31,0.59,48.60,X40N,NTT,MISC
83,TITAGARH,1097.23,1548.00,-3.47,H-SC,98.95,169645.0,-22370.0,101261.0,6.19,-11.65,59.69,41.08,95.0,-0.22,1.34,43.58,XY24,NTT,ENGINEERING
79,TANLA,917.30,1963.11,-38.20,H-SC,97.89,151785.0,-87630.0,360596.0,1.75,-36.60,237.57,114.01,20.0,-0.24,1.20,39.21,AR,ATH,IT
93,WHIRLPOOL,1151.42,2270.00,-34.28,M-SC,96.84,165570.0,19340.0,122720.0,0.56,13.23,74.12,97.15,27.0,0.16,1.31,41.45,X40,NTT,DURABLES
75,SONACOMS,578.05,1006.00,-27.37,M-MC,95.79,93844.0,-7315.0,82207.0,-1.22,-7.23,87.60,74.03,49.0,-0.09,0.74,31.89,AR,BTT,AUTO


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,17.17
1,25,38.74
2,50,68.16


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.94
X40N    15.45
X40     14.32
AR      14.15
XR      10.53
XY25     8.30
X200     1.19
SR       1.12
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.86
M-SC    17.96
X-LC    16.10
H-MC     9.03
M-LC     8.33
H-LC     5.94
X-MC     5.56
M-MC     5.56
X-SC     2.86
L-SC     2.31
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,11.73,0.33,31.38
BANKS,11.00,-7.94,60.88
IT,9.17,-18.97,85.34
FINANCE,7.57,-15.23,67.23
MISC,7.20,-23.87,77.17
INSURANCE,6.16,1.27,28.49
PAINTS,5.57,-16.42,50.87
AUTO,4.40,-8.25,55.43
ELECTRICAL,4.11,-6.52,60.79


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2918418.0,27
AR,1268707.0,15
XR,1210861.0,15
X40N,1201147.0,18
X40,778107.0,12
SR,202337.0,2
XY25,116395.0,5
X200,66289.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2288847.0,22
M-SC,1959152.0,21
X-LC,698448.0,12
M-MC,562735.0,5
H-MC,461109.0,8
X-SC,345521.0,4
M-LC,322955.0,6
L-SC,305644.0,4
X-MC,299891.0,5


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      881102.0      6
M-SC       XY24      740950.0      6
H-SC       AR        531298.0      3
           X40N      513595.0      7
M-SC       X40N      434953.0      6
           XR        416310.0      5
M-MC       XY24      397548.0      3
X-LC       X40       378295.0      5
H-SC       XR        276792.0      4
M-LC       XY24      202574.0      4
M-SC       X40       192436.0      2
H-LC       AR        190779.0      4
X-LC       AR        182153.0      3
H-MC       XY24      167572.0      2
L-SC       XR        160378.0      2
X-MC       XY24      146483.0      2
H-MC       AR        128520.0      2
X-SC       SR        118640.0      1
           XR        113680.0      1
           X40N      113201.0      2
M-LC       XR        103010.0      1
X-MC       X40N       99761.0      2
M-SC       AR         90806.0      1
H-SC       X40        86060.0      2
M-SC       SR         83697.0      1
M-MC       XR         82980.0      1
L-SC       XY24       82322.0      1
M-MC       AR         82207.0      1
X-LC       XY25       71711.0      3
H-MC       X40        67669.0      2
X-LC       X200       66289.0      1
L-SC       AR         62944.0      1
H-MC       XR         57711.0      1
X-MC       X40        53647.0      1
H-MC       X40N       39637.0      1
H-LC       XY25       27313.0      1
M-LC       XY25       17371.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 62.0 seconds
